In [1]:
from flask import Flask, render_template, request, redirect, url_for, session #Needed for web server
import csv  #Needed for reading/saving the data
import os  #Needed for reading/saving the data
import random #Needed for randomizing the order of the questions

In [2]:
app = Flask(__name__)
app.secret_key = os.urandom(24)

In [3]:
import csv
import random

def load_questions_from_csv(file_path):

    NUMBER_OF_QUESTIONS_PER_VALUE = 2
    
    # Initialize an empty list to store the questions
    questions = []

    # Initialize a set to store unique main values
    main_values = set()

    # Open the CSV file
    with open(file_path, 'r') as file:
        # Create a CSV reader object with '@' as the delimiter
        # The CSV should have the following columns:
        # - questionID
        # - main_value
        # - sub_value
        # - story
        reader = csv.DictReader(file, delimiter="@")
        
        # Iterate over each row in the CSV file
        for row in reader:
            # Append the row (question) to the list of questions
            questions.append(row)
            
            # Add the main value of the row to the set of main values
            main_values.add(row['main_value'])

    # Randomly select two questions for each main value
    selected_questions = []
    
    # Iterate over each main value
    for value in main_values:
        # Filter the questions based on the current main value
        value_questions = [q for q in questions if q['main_value'] == value]
        
        # Randomly select one question from the filtered questions
        selected = random.sample(value_questions, NUMBER_OF_QUESTIONS_PER_VALUE)
        
        # Add the selected question to the list of selected questions
        selected_questions.extend(selected)

    # Return the list of selected questions
    return selected_questions


In [4]:
def save_survey_results_to_csv(file_path, results):
    fieldnames = ['question_id', 'conservation', 'self_transcendence', 'openness_to_change', 'self_enhancement']

    # Check if the file already exists
    file_exists = os.path.isfile(file_path)

    with open(file_path, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # Write the header only if the file doesn't exist
        if not file_exists:
            writer.writeheader()

        writer.writerows(results)

Below is the code for the survey, in this version 4 values are tracked and recorded for the survey
    Conservation
    Openness to change
    Self Enhancement
    Self Transcendence

    To change the values the templates/survey.html page must also be changed

In [ ]:
# Route for the survey page
@app.route('/', methods=['GET', 'POST'])
def survey():
    # Check if 'current_question' is not in the session and initialize it to 0
    if 'current_question' not in session:
        session['current_question'] = 0

    # Check if 'survey_results' is not in the session and initialize it as an empty list
    if 'survey_results' not in session:
        session['survey_results'] = []

    # Load questions from CSV file
    questions = load_questions_from_csv('question_bank/questions.csv')

    # Get the current question and survey results from the session
    current_question = session['current_question']
    survey_results = session['survey_results']

    if request.method == 'POST':

        # Get the values from the submitted form
        question_id = request.form['question_id']
        conservation_rating = request.form['conservation']
        self_transcendence_rating = request.form['self_transcendence']
        openness_to_change_rating = request.form['openness_to_change']
        self_enhancement_rating = request.form['self_enhancement']

        # Store the question_id and ratings in the survey_results list
        survey_results.append({
            'question_id': question_id,
            'conservation': conservation_rating,
            'self_transcendence': self_transcendence_rating,
            'openness_to_change': openness_to_change_rating,
            'self_enhancement': self_enhancement_rating
        })

        if current_question == len(questions) - 1:
            # If the last question has been answered, save the survey results to a CSV file,
            # remove survey_results from the session, and redirect to the thank you page
            save_survey_results_to_csv('survey_results.csv', survey_results)
            session.pop('survey_results')
            return redirect(url_for('thank_you'))
        else:
            # After processing the submission, increment the current question and redirect back to the survey page
            session['current_question'] += 1
            return redirect(url_for('survey'))

    else:
        if current_question < 0:
            # Ensure current_question is not negative
            current_question = 0
        elif current_question >= len(questions):
            # If the current question exceeds the question count, redirect to the last question
            return redirect(url_for('survey'))

        # Retrieve the current question
        question = questions[current_question]
        print(question)

        # Render the survey page with the current question
        return render_template('survey.html', random_question=question)


In [6]:
@app.route('/thank_you')
def thank_you():
    return render_template('thank_you.html', survey_results=session.get('survey_results', []))


In [7]:
if __name__ == "__main__":
  app.debug = False
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jul/2023 13:08:07] "GET / HTTP/1.1" 200 -


{'id': '12', 'main_value': 'openness_to_change', 'sub_value': 'Stimulation', 'story': "John had been working in the sales office for over three years. He had started out as a junior salesperson, but had worked his way up to be one of the top performers in the office. He was always looking for new ways to make sales, and he was always looking for different things to do. He felt that it was important to stay ahead of the curve and to be able to adapt to changing markets. He was always looking for new ideas and ways to make sales. John had a lot of success in the office. He was always looking for new ways to make sales and he was always willing to try new things. He was always willing to take risks and to experiment with different strategies. He was always looking for ways to improve and to make the most of the opportunities that were presented to him. John's colleagues respected him for his willingness to try new things. They knew that he was always looking for different things to do and